In [1]:
from utils import *

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Bunny point cloud
path_to_bunny = "D:\\Skole\\Semester 10\\Prosjektoppgave\\Data\\bunny\\data\\bun000.ply"
bunny_ply = o3d.io.read_point_cloud(path_to_bunny)
bunny = np.asarray(bunny_ply.points)
bunny_true = np.copy(bunny)
n = bunny.shape[0]

In [3]:
# Correct rotation
rot = np.array([np.pi/3,
              np.pi/3,
              np.pi/3
            ])
# Noise and outliers
noise_1 = np.array([np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05)
                    ])
noise_2 = np.array([np.pi/3 + np.random.normal(0,0.5),
                    np.pi/3 + np.random.normal(0,0.5),
                    np.pi/3 + np.random.normal(0,0.5)
                    ])
outlier = np.array([np.pi*np.random.rand(),
                    np.pi*np.random.rand(),
                    np.pi*np.random.rand()
                    ])

In [4]:

# Generating translated cloud with inaccuracies
for i, p in enumerate(bunny):
    if i % 4 == 0:
        bunny[i] = p @ expso3(rot)
        continue
    if i % 4 == 1:
        bunny[i] = p @ expso3(noise_1)
        noise_1 = np.array([np.pi/3 + np.random.normal(0,0.005),
                    np.pi/3 + np.random.normal(0,0.005),
                    np.pi/3 + np.random.normal(0,0.005)
                    ])
        continue
    if i % 4 == 2:
        bunny[i] = p @ expso3(noise_2)
        noise_2 = np.array([np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05)
                    ])
        continue
    if i % 4 == 3:
        bunny[i] = p @ expso3(noise_2)
        outlier = np.array([np.pi*np.random.rand(),
                    np.pi*np.random.rand(),
                    np.pi*np.random.rand()
                    ])
        continue

In [5]:
# GNC Initialization
# Initial rotation guess
R0 = np.identity(3)

# Initial loss function data
r = np.zeros(n)
for i in range(n):
    r[i] = np.linalg.norm(bunny_true[i] - R0@bunny[i])
r0_max = np.max(r)

# gnc
eps = 0.011 
mu_update_factor = 1.4
max_iterations = 1000
w = np.ones(n)
mu = eps**2 / (2*r0_max**2 - eps**2)

In [6]:
last_iter = []
R_iter = [np.sum(r)]
iterations = 0
for i in range(max_iterations):
    iterations += 1
    last_iter.append(np.sum(w))
    # Weighted Procrustes
    H = bunny.T @ np.diag(w) @ bunny_true
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ np.diag([1,1,np.linalg.det( Vt.T @ U.T)]) @ U.T

    # Loss function
    for j in range(n):
        r[j] = np.linalg.norm(bunny_true[j] - R0@bunny[j])
        w[j] = w_from_r(r[j], eps, mu)

    R_iter.append(np.sum(r))

    mu = mu_update_factor * mu

    if i >= 5:
        if np.sum(w) == last_iter[i]:
            break

In [7]:
print(R)
print(expso3(rot))

[[ 0.86 -0.32  0.39]
 [ 0.42  0.89 -0.20]
 [-0.29  0.33  0.90]]
[[ 0.17 -0.15  0.97]
 [ 0.97  0.17 -0.15]
 [-0.15  0.97  0.17]]


In [8]:
inliers = []
outliers = []

for i, n in enumerate(w):
    if w[i] == 1.0:
        inliers.append(n)
    else:
        outliers.append(n)

percentage = (1-len(inliers)/(len(outliers)+len(inliers)))*100

print("Inliers:\t", len(inliers), "\nOutliers:\t", len(outliers),\
    "\nPercentage:\t",percentage, "%")

Inliers:	 7 
Outliers:	 40249 
Percentage:	 99.98261128775835 %


In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(bunny)
o3d.visualization.draw_geometries([pcd])

In [10]:
print(last_iter)

[40256.0, 85.12652629857374, 95.84132139374627, 106.57020336803068, 116.5893891477616, 125.20254772850652, 132.91013513536598, 138.62800166901525, 141.8257601080672, 142.60374917101947, 142.31335583742987, 141.26889107719046, 138.77071582030695, 134.10578174484255, 126.98170650942029, 117.58929559392283, 106.31565835835673, 93.35201164269637, 79.42930883996013, 65.38039556985446, 51.28840216621143, 37.759224162237274, 25.575523224316502, 15.914755145585236, 10.368165937317158, 8.055788091157986, 7.086971141251437, 6.693439938758633, 6.314142563425876, 6.163674854316109, 6.381757739808959, 6.425172212254445, 6.413984863160685, 6.393331612737569, 6.505217779550733, 6.709146715727911, 6.870935695150806, 7.0]
